# Question Processing System Demo

This notebook demonstrates our enhanced question processing system that uses:
- In-memory Milvus for vector similarity search
- Ollama for LLM-based question rewriting
- spaCy for natural language processing
- Sentence Transformers for text embeddings

In [1]:
# Import our question processor
from question_processor import InMemoryQuestionProcessor

# Create a processor instance
processor = InMemoryQuestionProcessor()
print("Processor initialized successfully!")

/Users/falcon/anaconda3/envs/question_extract_310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/falcon/seo_python/question_extract/src/question_processor.py:36: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  self.llm = Ollama(model=ollama_model, base_url=ollama_host)


Processor initialized successfully!


## 1. Question Extraction

Let's start by extracting questions from a sample text.

In [2]:
sample_text = """
Welcome to our AI and Machine Learning FAQ!

What is artificial intelligence? AI is a broad field of computer science.
How does machine learning work? It's based on patterns in data.
Can you explain neural networks in simple terms? Think of them as layered
information processors. Would you like to know more about deep learning?

The field is rapidly evolving. When will AI surpass human intelligence?
This remains a topic of debate.
"""

questions = processor.extract_questions(sample_text)
print("Extracted questions:")
for i, q in enumerate(questions, 1):
    print(f"{i}. {q}")

Extracted questions:
1. What is artificial intelligence?
2. How does machine learning work?
3. Can you explain neural networks in simple terms?
4. Would you like to know more about deep learning?
5. When will AI surpass human intelligence?


## 2. Adding Questions to the Store

Now let's add some questions to our vector store with categories.

In [3]:
# Add some AI-related questions
ai_questions = [
    "What is the role of backpropagation in neural networks?",
    "How do transformers process sequential data?",
    "What are the main types of machine learning?",
    "Can you explain how deep learning differs from traditional ML?"
]

for question in ai_questions:
    question_id = processor.add_question(question, category="ai")
    print(f"Added question (ID: {question_id}): {question}")

Added question (ID: 3): What is the role of backpropagation in neural networks?
Added question (ID: 4): How do transformers process sequential data?
Added question (ID: 5): What are the main types of machine learning?
Added question (ID: 6): Can you explain how deep learning differs from traditional ML?


## 3. Finding Similar Questions

Let's test the similarity search functionality.

In [4]:
test_question = "How does backpropagation work in neural nets?"
similar = processor.find_similar_questions(test_question, threshold=0.7)

print(f"Finding questions similar to: '{test_question}'\n")
for match in similar:
    print(f"Score: {match['score']:.2f} - {match['question']}")

Finding questions similar to: 'How does backpropagation work in neural nets?'

Score: 0.90 - What is the role of backpropagation in neural networks?


## 4. Question Style Rewriting

Let's see how our system can rewrite questions in different styles using Ollama.

In [5]:
questions_to_rewrite = [
    "Hey, can you tell me what machine learning is?",
    "Would you be so kind as to explain the concept of neural networks?"
]

for question in questions_to_rewrite:
    print(f"\nOriginal: {question}")
    print(f"Formal: {processor.rewrite_question(question, 'formal')}")
    print(f"Casual: {processor.rewrite_question(question, 'casual')}")


Original: Hey, can you tell me what machine learning is?


/Users/falcon/seo_python/question_extract/src/question_processor.py:115: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return self.llm.predict(prompt).strip()


Formal: Certainly! Here is a rewritten version of the question in a more formal style:

"Could you provide an explanation regarding the concept of machine learning? Specifically, could you elaborate on the nature and applications of this field, as well as any relevant theoretical foundations or methodological approaches?"
Casual: Yo, can you spill the deets on this machine learning thingy? Like, what is it even do? 🤔

Original: Would you be so kind as to explain the concept of neural networks?
Formal: Sure, I would be delighted to elucidate the notion of neural networks for you. Neural networks are a type of machine learning model that are composed of interconnected nodes or "neurons," which process and transmit information in a manner reminiscent of the human brain. These networks are capable of learnings complex patterns and relationships within data, and have been successfully applied to a diverse array of tasks, including image recognition, natural language processing, and predicti

## 5. Complete Document Processing

Finally, let's process a complete document that combines all these features.

In [6]:
technical_doc = """
Deep Learning and Neural Networks: A Primer

What are neural networks? They are computational models inspired by biological
neural networks. How do they learn from data? Through a process called
backpropagation and gradient descent.

Would you please explain the different types of neural networks? There are
several types including CNNs, RNNs, and Transformers. Hey, can anyone tell
me the difference between supervised and unsupervised learning?
"""

results = processor.process_document(technical_doc)

print("Processed Document Results:\n")
for i, result in enumerate(results['processed_questions'], 1):
    print(f"Question {i}:")
    print(f"Original: {result['original']}")
    
    if result['similar']:
        print("Similar questions found:")
        for match in result['similar']:
            print(f"- {match['score']:.2f}: {match['question']}")
    
    print("Variations:")
    print(f"- Formal: {result['variations']['formal']}")
    print(f"- Casual: {result['variations']['casual']}")
    print()

Processed Document Results:

Question 1:
Original: Deep Learning and Neural Networks: A Primer

What are neural networks?
Variations:
- Formal: Neural Networks: An Overview

What are neural networks, precisely? In essence, they are computational models inspired by the structure and function of the human brain. These models are composed of interconnected nodes or neurons that process information through a system of weights and biases. The connections between neurons enable the network to learn from data and adapt its internal parameters to perform various tasks, such as classification, regression, and feature extraction.

In simple terms, neural networks are made up of layers of artificial neurons that work together to analyze and solve complex problems. Each layer processes a different aspect of the input data, allowing the network to extract valuable information and make accurate predictions or decisions. By adjusting the weights and biases of these connections, the network can learn 

## 6. Cleanup

Let's properly clean up our resources.

In [7]:
# The processor will automatically clean up in its destructor,
# but we can do it explicitly
del processor
print("Resources cleaned up successfully!")

Resources cleaned up successfully!
